In [1]:
import os

# 禁用 JAX 的显存预分配
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

import jax
from evox import Problem, algorithms,jit_class, monitors, workflows
from evox.operators import mutation, crossover, selection
from jax import random
from dataclasses import dataclass
import jax.numpy as jnp

import sys
sys.path.append('../my_es_algorithm')  # 添加路径到系统路径

In [2]:
## 产电机组参数
unit_idp = jnp.arange(1, 14)
alpha = jnp.array([0.00028, 0.00056, 0.00056, 0.00324, 0.00324, 0.00324, 0.00324, 0.00324, 0.00324, 0.00284, 0.00284, 0.00284, 0.00284])
beta = jnp.array([8.1, 8.1, 8.1, 7.74, 7.74, 7.74, 7.74, 7.74, 7.74, 8.6, 8.6, 8.6, 8.6])
gamma = jnp.array([550, 309, 309, 240, 240, 240, 240, 240, 240, 126, 126, 126, 126])
lambda_vals = jnp.array([300, 200, 200, 150, 150, 150, 150, 150, 150, 100, 100, 100, 100])
rho = jnp.array([0.035, 0.042, 0.042, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.084, 0.084, 0.084, 0.084])
Pmin = jnp.array([0, 0, 0, 60, 60, 60, 60, 60, 60, 40, 40, 55, 55])
Pmax = jnp.array([680, 360, 360, 180, 180, 180, 180, 180, 180, 120, 120, 120, 120])

# Compiling into a dictionary for structured display or usage
power_units = {
    "Unit": unit_idp,
    "Alpha": alpha,
    "Beta": beta,
    "Gamma": gamma,
    "Lambda": lambda_vals,
    "Rho": rho,
    "Pmin": Pmin,
    "Pmax": Pmax
}



In [3]:
### 如果项目中的机组参数需要更细粒度的操作，且每个实例的属性独立操作较多，@dataclass 会提高代码可读性和扩展性
# # from dataclasses import dataclass
# # import jax.numpy as jnp

# @dataclass
# class PowerUnit:
#     unit_id: int
#     alpha: float
#     beta: float
#     gamma: float
#     lambda_val: float
#     rho: float
#     Pmin: float
#     Pmax: float

# # 初始化每个机组的参数
# unit_ids = jnp.arange(1, 14)
# alphas = jnp.array([0.00028, 0.00056, 0.00056, 0.00324, 0.00324, 0.00324, 0.00324, 0.00324, 0.00324, 0.00284, 0.00284, 0.00284, 0.00284])
# betas = jnp.array([8.1, 8.1, 8.1, 7.74, 7.74, 7.74, 7.74, 7.74, 7.74, 8.6, 8.6, 8.6, 8.6])
# gammas = jnp.array([550, 309, 309, 240, 240, 240, 240, 240, 240, 126, 126, 126, 126])
# lambda_vals = jnp.array([300, 200, 200, 150, 150, 150, 150, 150, 150, 100, 100, 100, 100])
# rhos = jnp.array([0.035, 0.042, 0.042, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.084, 0.084, 0.084, 0.084])
# Pmins = jnp.array([0, 0, 0, 60, 60, 60, 60, 60, 60, 40, 40, 55, 55])
# Pmaxs = jnp.array([680, 360, 360, 180, 180, 180, 180, 180, 180, 120, 120, 120, 120])

# # 创建机组列表
# power_units = [
#     PowerUnit(unit_id, alpha, beta, gamma, lambda_val, rho, Pmin, Pmax)
#     for unit_id, alpha, beta, gamma, lambda_val, rho, Pmin, Pmax in zip(
#         unit_ids, alphas, betas, gammas, lambda_vals, rhos, Pmins, Pmaxs
#     )
# ]

# # 示例：输出所有机组的参数
# for unit in power_units:
#     print(unit)


In [4]:
## 热点联产机组参数
unit_idhp = jnp.array([14, 15, 16, 17, 18, 19])
a = jnp.array([0.0345, 0.0435, 0.0345, 0.0435, 0.1035, 0.072])
b = jnp.array([14.5, 36, 14.5, 36, 34.5, 20])
c = jnp.array([2650, 1250, 2650, 1250, 2650, 1565])
d = jnp.array([0.03, 0.027, 0.03, 0.027, 0.025, 0.02])
e = jnp.array([4.2, 0.6, 4.2, 0.6, 2.203, 2.34])
f = jnp.array([0.031, 0.011, 0.031, 0.011, 0.051, 0.04])
PRmin = jnp.array([81, 40, 81, 40, 10, 35])
PRmax = jnp.array([247, 125.8, 247, 125.8, 60, 105])
HRmin = jnp.array([0, 0, 0, 0, 0, 0])
HRmax = jnp.array([180, 135.6, 180, 135.6, 55, 45])

# Feasible region coordinates for Pc and Hc
feasible_region = {
    14: jnp.array([[98.8, 0], [81, 104.8], [215, 180], [247, 0]]),
    15: jnp.array([[44, 0], [44, 15.9], [40, 75], [110.2, 135.6], [125.8, 32.4], [125.8, 0]]),
    16: jnp.array([[98.8, 0], [81, 104.8], [215, 180], [247, 0]]),
    17: jnp.array([[44, 0], [44, 15.9], [40, 75], [110.2, 135.6], [125.8, 32.4], [125.8, 0]]),
    18: jnp.array([[20, 0], [10, 40], [45, 55], [60, 0]]),
    19: jnp.array([[35, 0], [35, 20], [90, 45], [90, 25], [105, 0]])
}

# Combine into a structured dictionary
chp_units = {
    "Units": unit_idhp,
    "a": a,
    "b": b,
    "c": c,
    "d": d,
    "e": e,
    "f": f,
    "Feasible_Region": feasible_region,
    # 添加缺失的键
    "PRmin": PRmin,
    "PRmax": PRmax,
    "HRmin": HRmin,
    "HRmax": HRmax
}

In [5]:
## 产热机组参数
# Heat-only unit parameters
unit_idh = jnp.array([20, 21, 22, 23, 24])
a_heat = jnp.array([0.038, 0.038, 0.038, 0.052, 0.052])
b_heat = jnp.array([2.0109, 2.0109, 2.0109, 3.0651, 3.0651])
c_heat = jnp.array([950, 950, 950, 480, 480])
Hmin = jnp.array([0, 0, 0, 0, 0])
Hmax = jnp.array([2695.2, 60, 60, 120, 120])

# Combine into a structured dictionary
heat_units = {
    "Units": unit_idh,
    "a": a_heat,
    "b": b_heat,
    "c": c_heat,
    "Hmin": Hmin,
    "Hmax": Hmax
}

In [6]:
# def adjust_population_random_loop(x, Pd, key, tolerance):  # 多次调整
#     n = x.shape[0]  # 获取发电机的数量

#     def cond_fun(state):
#         # x, sumpop, df, r = state
#         x, sumpop = state
#         return jnp.abs(sumpop - Pd) >= tolerance

#     def body_fun(state):
#         x, sumpop = state
#         df = sumpop - Pd  # 计算功率偏差
#         r = random.randint(key, shape=(), minval=0, maxval=n)  # 生成随机数
#         x = x.at[r].set(x[r] - df)  # 使用 .at[].set() 更新特定索引的值
#         # x = jnp.clip(x, lb, ub)  # 确保更新后的值不超出上下限
#         sumpop = jnp.sum(x)  # 计算更新后所有发电机功率的总和
#         return x, sumpop

#     state = (x, jnp.sum(x))
#     x, _= jax.lax.while_loop(cond_fun, body_fun, state)

#     # 新增：将最终结果保留两位小数
#     x = jnp.round(x, 2)
    
#     return x

In [7]:
Pd = 2350
Hd = 1250
dim_P = 13
dim_PH = 6
dim_H = 5

In [8]:
from jax import lax
def update_chp_unit_heat(power_chp, heat):
        """
        更新热电联产机组的热能输出

        参数:
            power_chp: 热电联产机组的功率输出数组 (dim_PH,)
            heat: 热能输出数组 (dim_PH,)

        返回:
            更新后的热能输出数组
        """
        # 将功率和热能输出转换为 float32 类型
        p = power_chp.astype(jnp.float32)
        h = heat.astype(jnp.float32)

        # 定义计算热能输出的辅助函数
        def compute_heat(hi, lower_bound, upper_bound):
            return jnp.clip(hi, lower_bound, upper_bound)

        # 定义每个索引对应的处理逻辑
        def compute_adjusted_heat(index):
            pi = p[index]
            hi = h[index]

            def chp_14_16():
                cond1 = (pi >= 81) & (pi <= 98.8)
                cond2 = (pi > 98.8) & (pi <= 215)
                cond3 = (pi > 215) & (pi <= 247)

                lower_bound = jnp.where(cond1, 104.8 - (524 / 89) * (pi - 81), 0.0)
                upper_bound = jnp.where(
                    cond1 | cond2,
                    104.8 + (188 / 335) * (pi - 81),
                    jnp.where(cond3, - (45 / 8) * (pi - 247), 0.0)
                )
                return compute_heat(hi, lower_bound, upper_bound)

            def chp_15_17():
                cond1 = (pi >= 40) & (pi <= 44)
                cond2 = (pi > 44) & (pi <= 110.2)
                cond3 = (pi > 110.2) & (pi <= 125.8)

                lower_bound = jnp.where(cond1, 75 - (591 / 40) * (pi - 40), 0.0)
                upper_bound = jnp.where(
                    cond1 | cond2,
                    75 + (101 / 117) * (pi - 40),
                    jnp.where(cond3, 32.4 - (86 / 13) * (pi - 125.8), 0.0)
                )
                return compute_heat(hi, lower_bound, upper_bound)

            def chp_18():
                cond1 = (pi >= 10) & (pi <= 20)
                cond2 = (pi > 20) & (pi <= 45)
                cond3 = (pi > 45) & (pi <= 60)

                lower_bound = jnp.where(cond1, 40 - 4 * (pi - 10), 0.0)
                upper_bound = jnp.where(
                    cond1 | cond2,
                    40 + (3 / 7) * (pi - 10),
                    jnp.where(cond3, - (11 / 3) * (pi - 60), 0.0)
                )
                return compute_heat(hi, lower_bound, upper_bound)

            def chp_19():
                cond1 = (pi >= 35) & (pi <= 90)
                cond2 = (pi > 90) & (pi <= 105)

                lower_bound = 0.0
                upper_bound = jnp.where(
                    cond1,
                    20 + (5 / 11) * (pi - 35),
                    jnp.where(cond2, - (5 / 3) * (pi - 105), 0.0)
                )
                return compute_heat(hi, lower_bound, upper_bound)

            def default_case():
                return hi

            # 使用 lax.cond 选择对应的处理逻辑
            adjusted_hi = lax.cond(
                (index == 0) | (index == 2),
                chp_14_16,
                lambda: lax.cond(
                    (index == 1) | (index == 3),
                    chp_15_17,
                    lambda: lax.cond(
                        index == 4,
                        chp_18,
                        lambda: lax.cond(
                            index == 5,
                            chp_19,
                            default_case
                        )
                    )
                )
            )
            #return adjusted_hi

        # 使用 jax.vmap 对所有索引进行处理
        #adjusted_heat = jax.vmap(compute_adjusted_heat)(jnp.arange(dim_PH))
        adjusted_heat = compute_adjusted_heat(jnp.arange(dim_PH))
        return adjusted_heat

In [9]:
def update_chp_unit_heat_all(power_chp, heat):
        """
        更新热电联产机组的热能输出

        参数:
            power_chp: 热电联产机组的功率输出数组 (dim_PH,)
            heat: 热能输出数组 (dim_PH,)

        返回:
            更新后的热能输出数组
        """
        # 将功率和热能输出转换为 float32 类型
        p = power_chp.astype(jnp.float32)
        h = heat.astype(jnp.float32)

        # 定义机组类型的掩码
        indices = jnp.arange(dim_PH)

         # 创建掩码
        mask_14_16 = (indices == 0) | (indices == 2)
        mask_15_17 = (indices == 1) | (indices == 3)
        mask_18 = (indices == 4)
        mask_19 = (indices == 5)
        #mask_default = ~(mask_14_16 | mask_15_17 | mask_18 | mask_19)

         # 扩展掩码以匹配形状
        mask_14_16 = mask_14_16[jnp.newaxis, :]
        mask_15_17 = mask_15_17[jnp.newaxis, :]
        mask_18 = mask_18[jnp.newaxis, :]
        mask_19 = mask_19[jnp.newaxis, :]

        # 初始化 adjusted_heat
        adjusted_heat = jnp.zeros_like(heat)

        # 计算 chp_14_16 的 lower_bound 和 upper_bound
        cond1_14_16 = (p >= 81) & (p <= 98.8)
        cond2_14_16 = (p > 98.8) & (p <= 215)
        cond3_14_16 = (p > 215) & (p <= 247)

        lower_bound_14_16 = jnp.where(cond1_14_16, 104.8 - (524 / 89) * (p - 81), 0.0)
        upper_bound_14_16 = jnp.where(
            cond1_14_16 | cond2_14_16,
            104.8 + (188 / 335) * (p - 81),
            jnp.where(cond3_14_16, - (45 / 8) * (p - 247), 0.0)
        )
        adjusted_14_16 = jnp.clip(h, lower_bound_14_16, upper_bound_14_16)
        adjusted_heat = jnp.where(mask_14_16, adjusted_14_16, adjusted_heat)

        # 计算 chp_15_17 的 lower_bound 和 upper_bound
        cond1_15_17 = (p >= 40) & (p <= 44)
        cond2_15_17 = (p > 44) & (p <= 110.2)
        cond3_15_17 = (p > 110.2) & (p <= 125.8)

        lower_bound_15_17 = jnp.where(cond1_15_17, 75 - (591 / 40) * (p - 40), 0.0)
        upper_bound_15_17 = jnp.where(
            cond1_15_17 | cond2_15_17,
            75 + (101 / 117) * (p - 40),
            jnp.where(cond3_15_17, 32.4 - (86 / 13) * (p - 125.8), 0.0)
        )
        adjusted_15_17 = jnp.clip(h, lower_bound_15_17, upper_bound_15_17)
        adjusted_heat = jnp.where(mask_15_17, adjusted_15_17, adjusted_heat)

        # 计算 chp_18 的 lower_bound 和 upper_bound
        cond1_18 = (p >= 10) & (p <= 20)
        cond2_18 = (p > 20) & (p <= 45)
        cond3_18 = (p > 45) & (p <= 60)

        lower_bound_18 = jnp.where(cond1_18, 40 - 4 * (p - 10), 0.0)
        upper_bound_18 = jnp.where(
            cond1_18 | cond2_18,
            40 + (3 / 7) * (p - 10),
            jnp.where(cond3_18, - (11 / 3) * (p - 60), 0.0)
        )
        adjusted_18 = jnp.clip(h, lower_bound_18, upper_bound_18)
        adjusted_heat = jnp.where(mask_18, adjusted_18, adjusted_heat)
        
        # 计算 chp_19 的 lower_bound 和 upper_bound
        cond1_19 = (p >= 35) & (p <= 90)
        cond2_19 = (p > 90) & (p <= 105)

        lower_bound_19 = 0.0
        upper_bound_19 = jnp.where(
            cond1_19,
            20 + (5 / 11) * (p - 35),
            jnp.where(cond2_19, - (5 / 3) * (p - 105), 0.0)
        )
        adjusted_19 = jnp.clip(h, lower_bound_19, upper_bound_19)
        adjusted_heat = jnp.where(mask_19, adjusted_19, adjusted_heat)
        
        adjusted_heat = jnp.squeeze(adjusted_heat, axis=0)  # 去除第0维

        return adjusted_heat

In [10]:
def update_chp_unit_heat_once(pi, hi, r):
        """
        调整热能输出基于输入的 r 值。

        参数:
            pi: 功率输出 (标量或数组)
            hi: 热能输出 (标量或数组)
            r: 指定要进行哪种调整的标识符

        返回:
            调整后的热能输出
        """
        # 将 pi 和 hi 转换为 float32 类型
        pi = jnp.array(pi, dtype=jnp.float32)
        hi = jnp.array(hi, dtype=jnp.float32)

        # 定义计算热能输出的辅助函数
        def compute_heat(hi, lower_bound, upper_bound):
            return jnp.clip(hi, lower_bound, upper_bound)

        # 定义各个调整函数
        def chp_14_16():
            cond1 = (pi >= 81) & (pi <= 98.8)
            cond2 = (pi > 98.8) & (pi <= 215)
            cond3 = (pi > 215) & (pi <= 247)

            lower_bound = jnp.where(cond1, 104.8 - (524 / 89) * (pi - 81), 0.0)
            upper_bound = jnp.where(
                cond1 | cond2,
                104.8 + (188 / 335) * (pi - 81),
                jnp.where(cond3, - (45 / 8) * (pi - 247), 0.0)
            )
            return compute_heat(hi, lower_bound, upper_bound)

        def chp_15_17():
            cond1 = (pi >= 40) & (pi <= 44)
            cond2 = (pi > 44) & (pi <= 110.2)
            cond3 = (pi > 110.2) & (pi <= 125.8)

            lower_bound = jnp.where(cond1, 75 - (591 / 40) * (pi - 40), 0.0)
            upper_bound = jnp.where(
                cond1 | cond2,
                75 + (101 / 117) * (pi - 40),
                jnp.where(cond3, 32.4 - (86 / 13) * (pi - 125.8), 0.0)
            )
            return compute_heat(hi, lower_bound, upper_bound)

        def chp_18():
            cond1 = (pi >= 10) & (pi <= 20)
            cond2 = (pi > 20) & (pi <= 45)
            cond3 = (pi > 45) & (pi <= 60)

            lower_bound = jnp.where(cond1, 40 - 4 * (pi - 10), 0.0)
            upper_bound = jnp.where(
                cond1 | cond2,
                40 + (3 / 7) * (pi - 10),
                jnp.where(cond3, - (11 / 3) * (pi - 60), 0.0)
            )
            return compute_heat(hi, lower_bound, upper_bound)

        def chp_19():
            cond1 = (pi >= 35) & (pi <= 90)
            cond2 = (pi > 90) & (pi <= 105)

            lower_bound = 0.0
            upper_bound = jnp.where(
                cond1,
                20 + (5 / 11) * (pi - 35),
                jnp.where(cond2, - (5 / 3) * (pi - 105), 0.0)
            )
            return compute_heat(hi, lower_bound, upper_bound)

        def default_case():
            return hi

        # 根据 r 的值选择对应的调整函数
        adjusted_hi = lax.cond(
            (r == 0) | (r == 2),
            chp_14_16,
            lambda: lax.cond(
                (r == 1) | (r == 3),
                chp_15_17,
                lambda: lax.cond(
                    r == 4,
                    chp_18,
                    lambda: lax.cond(
                        r == 5,
                        chp_19,
                        default_case
                    )
                )
            )
        )

        return adjusted_hi

In [11]:
def adjust_power_balance(solution):
        """调整功率输出以满足功率等式约束"""
        # 定义解数组中功率部分的索引
        power_indices = slice(0, dim_P + dim_PH)
        power_chp_indices = slice(dim_P, dim_P + dim_PH)
        heat_chp_indices = slice(dim_P + dim_PH, dim_P + 2 * dim_PH)
        
        # 计算当前总功率与目标功率的差值
        total_power = solution[power_indices]
        delta_power = jnp.sum(total_power) - Pd

        #key = jax.random.PRNGKey(0)
         # 定义条件函数，检查 delta_power 是否大于阈值
        def condition(state):
            delta_power, power = state
            return jnp.abs(delta_power) > 1e-2
        
        # 定义循环体函数
        def body(state):
            delta_power, power = state
            
            # 计算一个调整量而不是每次都选择使用这个差值进行计算
            adjustment = delta_power / (dim_P + dim_PH)
            power = power - adjustment

            # 第一部分：dim_P 元素
            power_P = power[:dim_P]
            power_P = jnp.clip(power_P, Pmin, Pmax)
            # 第二部分：dim_PH 元素
            power_PH = power[dim_P:]
            power_PH = jnp.clip(power_PH, PRmin, PRmax)
            # 合并调整后的功率部分
            power = jnp.concatenate([power_P, power_PH])
            # 重新计算 delta_power
            new_delta_power = jnp.sum(power) - Pd
            return new_delta_power, power
        
        # 获取初始热能状态
        initial_heat = solution[heat_chp_indices]

        ## 执行循环确保功率平衡
        delta_power, total_power = jax.lax.while_loop(
            condition,
            body,
            (delta_power, total_power)
        )
        ## 将更新后的功率写回解中
        #total_power = solution.at[power_indices].set(total_power)
        solution = solution.at[power_indices].set(total_power)

        ## 更新热能输出
        total_heat = update_chp_unit_heat_all(total_power[power_chp_indices], initial_heat)
        solution = solution.at[heat_chp_indices].set(total_heat)

        return solution




def adjust_heat_balance(solution):
        # 定义热能输出部分的索引
        heat_indices = slice(dim_P + dim_PH, None)
        # 定义热电联产机组热功率部分的索引
        heat_chp_indices = slice(dim_P + dim_PH, dim_P + 2 * dim_PH)
        # 定义纯热电机组热功率部分的索引
        heat_thermal_indices = slice(dim_P + 2 * dim_PH, None)


## 先根据之前热电机组的电功率来热功率输出
        power_chp_indices = slice(dim_P, dim_P + dim_PH)
        # 获取热电联产机组的功率输出
        power_chp = solution[power_chp_indices]
        heat_chp = solution[heat_chp_indices]
        # 对联合热电机组（热功率部分）应用热能约束
        update_heat_chp = update_chp_unit_heat_all(power_chp, heat_chp)
        #[考虑移除这步] 将更新后的热电联产机组热能输出重新赋值回原始的 solution 数组
        solution = solution.at[heat_chp_indices].set(update_heat_chp)

        
        # 获取所有的热能输出
        total_heat = solution[heat_indices]
        # 获取热电联产机组的热能输出
        #heat_chp = solution[heat_chp_indices]
        # 获取纯热电机组的热能输出
        heat_thermal = solution[heat_thermal_indices]

        # ## 先根据之前热电机组的电功率来热功率输出
        # power_chp_indices = slice(dim_P, dim_P + dim_PH)
        # # 获取热电联产机组的功率输出
        # power_chp = solution[power_chp_indices]

        # # 对联合热电机组（热功率部分）应用热能约束
        # update_heat_chp = update_chp_unit_heat_all(power_chp, heat_chp)
        # #[考虑移除这步] 将更新后的热电联产机组热能输出重新赋值回原始的 solution 数组
        # solution = solution.at[heat_chp_indices].set(update_heat_chp)

        ## 再循环调整满足热能平衡约束
        # 计算热能差值
        # delta_heat = jnp.sum(update_heat_chp) + jnp.sum(heat_thermal) - self.Hd
        total_heat = jnp.concatenate([update_heat_chp, heat_thermal])
        delta_heat = jnp.sum(total_heat) - Hd
        
         # 热能平衡约束循环
        def heat_condition(state):
            delta_heat, heat = state
            return jnp.abs(delta_heat) > 1e-2
        
        # 定义循环体函数
        def heat_body(state):
            delta_heat, heat = state
            # # 随机选择一个索引
            # key, subkey = jax.random.split(key)
            # r = jax.random.randint(subkey, shape=(), minval=0, maxval=self.dim_PH + self.dim_H)
            # 计算一个调整量
            adjustment_heat = delta_heat / (dim_PH + dim_H)

            # 调整热电联产机组
            heat_chp = heat[:dim_PH]
            power_chp = solution[power_chp_indices]
            indices_chp = jnp.arange(dim_PH)

            def adjust_chp_unit(p, h, idx):
                new_heat = h - adjustment_heat
                new_heat = update_chp_unit_heat_once(p, new_heat, idx)
                return new_heat

            # 向量化调整热电联产机组的热能输出
            heat_chp_updated = jax.vmap(adjust_chp_unit, in_axes=(0, 0, 0))(power_chp, heat_chp, indices_chp)

            # 调整纯热机组
            heat_thermal = heat[dim_PH : ]
            heat_thermal_adjusted = heat_thermal - adjustment_heat
            heat_thermal_updated = jnp.clip(heat_thermal_adjusted, Hmin, Hmax)

            # 合并更新后的热能输出
            heat_updated = jnp.concatenate([heat_chp_updated, heat_thermal_updated])

            # 重新计算 delta_heat
            delta_heat = jnp.sum(heat_updated) - Hd

            return delta_heat, heat_updated

        # 初始化循环状态
        state = (delta_heat, total_heat)

        # 执行循环，调整热能输出以满足热能平衡约束
        delta_heat, total_heat = jax.lax.while_loop(
            heat_condition,
            heat_body,
            state
        )

        # 更新 solution 中的热能输出部分
        solution = solution.at[heat_indices].set(total_heat)

        return solution

In [12]:
def adjust_balance(solution):
        """调整功率和热能输出以满足功率和热能平衡约束"""
        # 定义功率输出部分的索引
        power_indices = slice(0, dim_P + dim_PH)
        power_P_indices = slice(0, dim_P)
        power_PH_indices = slice(dim_P, dim_P + dim_PH)

        # 定义热能输出部分的索引
        heat_indices = slice(dim_P + dim_PH, None)
        heat_chp_indices = slice(dim_P + dim_PH, dim_P + 2 * dim_PH)
        heat_thermal_indices = slice(dim_P + 2 * dim_PH, None)

        ## 调整功率平衡约束
        total_power = solution[power_indices]
        delta_power = jnp.sum(total_power) - Pd

        # 定义功率平衡约束的条件函数
        def power_condition(state):
            delta_power, power = state
            return jnp.abs(delta_power) > 1e-2

        # 定义功率平衡约束的循环体函数
        def power_body(state):
            delta_power, power = state

            # 计算调整量
            adjustment = delta_power / (dim_P + dim_PH)
            power = power - adjustment

            # 调整功率单元的输出并进行剪切
            power_P = power[power_P_indices]
            power_P = jnp.clip(power_P, Pmin, Pmax)

            power_PH = power[power_PH_indices]
            power_PH = jnp.clip(power_PH, PRmin, PRmax)

            # 合并调整后的功率部分
            power = jnp.concatenate([power_P, power_PH])

            # 重新计算 delta_power
            new_delta_power = jnp.sum(power) - Pd

            return new_delta_power, power

        # 执行循环以确保功率平衡
        delta_power, total_power = jax.lax.while_loop(
            power_condition,
            power_body,
            (delta_power, total_power)
        )

        # 将更新后的功率写回解中
        solution = solution.at[power_indices].set(total_power)

        ## 调整热能平衡约束

        # 获取热电联产机组的功率输出
        power_chp = solution[power_PH_indices]

        # 获取热电联产机组的热能输出
        heat = solution[heat_indices]
        heat_chp = heat[:dim_PH]

        # 根据功率输出更新热电联产机组的热能输出
        heat_chp_updated = update_chp_unit_heat_all(power_chp, heat_chp)

        # 将更新后的热电联产机组热能输出写回解中
        solution = solution.at[heat_chp_indices].set(heat_chp_updated)
        heat = solution[heat_indices]

        # 获取所有的热能输出
        heat_thermal = heat[dim_PH:]
        total_heat = jnp.concatenate([heat_chp_updated, heat_thermal])

        delta_heat = jnp.sum(total_heat) - Hd

        # 定义热能平衡约束的条件函数
        def heat_condition(state):
            delta_heat, heat = state
            return jnp.abs(delta_heat) > 1e-2

        # 定义热能平衡约束的循环体函数
        def heat_body(state):
            delta_heat, heat = state
            adjustment_heat = delta_heat / (dim_PH + dim_H)

            # 调整热电联产机组的热能输出
            heat_chp = heat[:dim_PH]
            power_chp = solution[power_PH_indices]
            indices_chp = jnp.arange(dim_PH)

            def adjust_chp_unit(p, h, idx):
                new_heat = h - adjustment_heat
                new_heat = update_chp_unit_heat_once(p, new_heat, idx)
                return new_heat

            # 向量化调整热电联产机组的热能输出
            heat_chp_updated = jax.vmap(adjust_chp_unit, in_axes=(0, 0, 0))(power_chp, heat_chp, indices_chp)

            # 调整纯热机组的热能输出
            heat_thermal = heat[dim_PH:]
            heat_thermal_adjusted = heat_thermal - adjustment_heat
            heat_thermal_updated = jnp.clip(heat_thermal_adjusted, Hmin, Hmax)

            # 合并更新后的热能输出
            heat_updated = jnp.concatenate([heat_chp_updated, heat_thermal_updated])

            # 重新计算 delta_heat
            delta_heat = jnp.sum(heat_updated) - Hd

            return delta_heat, heat_updated

        # 初始化循环状态
        state = (delta_heat, total_heat)

        # 执行循环，调整热能输出以满足热能平衡约束
        delta_heat, total_heat = jax.lax.while_loop(
            heat_condition,
            heat_body,
            state
        )

        # 更新 solution 中的热能输出部分
        solution = solution.at[heat_indices].set(total_heat)

        return solution

In [13]:
def adjust_power_heat_limitation(solution):

        """对三种不同的机组应用功率和热能约束。"""
        # 定义解数组中不同部分的索引
        power_indices = slice(0, dim_P)
        power_chp_indices = slice(dim_P, dim_P + dim_PH)
        power_heat_indices = slice(dim_P + dim_PH, dim_P + 2 * dim_PH)
        thermal_indices = slice(dim_P + 2 * dim_PH, None)

        # 对纯电机组应用功率约束
        adjusted_power = jnp.clip(solution[power_indices], Pmin, Pmax)

        # 对联合热电机组（电功率部分）应用功率约束
        adjusted_power_chp = jnp.clip(solution[power_chp_indices], PRmin, PRmax)

        # 对联合热电机组（热功率部分）应用热能约束
        adjusted_power_heat = update_chp_unit_heat_all(adjusted_power_chp, solution[power_heat_indices])
        
        # 对纯热机组应用热能约束
        adjusted_thermal = jnp.clip(solution[thermal_indices], Hmin, Hmax)
        #adjusted_solution = adjusted_solution.at[thermal_indices].set(adjusted_thermal)
         # 合并所有调整后的部分
        adjusted_solution = jnp.concatenate([
            adjusted_power,
            adjusted_power_chp,
            adjusted_power_heat,
            adjusted_thermal
        ])
        return adjusted_solution

In [14]:
from typing import Tuple, Any
def _total_cost(x):
    """
    计算总成本，包括纯电机组成本、热点联产机组成本和产热机组成本。

    参数：
        x: 输入数组，形状为 (样本数, 30)

    返回：
        total_cost_array: 总成本数组，形状为 (样本数,)
    """
    # 确保输入是二维的
    # if x.ndim == 1:
    #     x = x[jnp.newaxis, :]
    x = adjust_power_heat_limitation(x)
    x = adjust_power_balance(x)
    x = adjust_heat_balance(x)
    #x = adjust_power_heat_limitation(x)
    #x = adjust_balance(x)
    # 提取输入数组的不同部分
    xp1 = x[0:dim_P]    # 纯电机组的功率输出
    xp2 = x[dim_P:dim_P + dim_PH]   # CHP机组的功率输出
    xh2 = x[dim_P + dim_PH:dim_P + 2 * dim_PH]   # CHP机组的热力输出
    xh3 = x[dim_P + 2 * dim_PH:None]   # 产热机组的热力输出

    # 计算纯电机组成本
    term1 = alpha * xp1**2 + beta * xp1 + gamma
    cost1 = term1.sum() + jnp.abs(lambda_vals * jnp.sin(rho * (xp1 - Pmin))).sum()
    
    # 使用 chp_units 中的参数
    a = chp_units['a']  # 形状为 (6,)
    b = chp_units['b']
    c = chp_units['c']
    d = chp_units['d']
    e = chp_units['e']
    f = chp_units['f']


    
    # 计算热点联产机组成本
    cost2 = (a * xp2**2 + b * xp2 + c + d * xh2 ** 2 + e * xh2 + f * xp2 * xh2).sum()
    
    # 计算产热机组成本
    cost3 = (a_heat * xh3**2 + b_heat * xh3 + c_heat).sum()
    
    # 计算总成本
    total_cost_array = cost1 + cost2 + cost3
    
    return total_cost_array


def total_cost(x):
    return  jax.vmap(_total_cost, in_axes= 0)(x)

@jit_class
class CHPELD_24(Problem):
    """
    CHPELD_24问题类,继承自Problem类,用于评估热电联产机组的总成本。
    """
    def __init__(self):
        super().__init__()
 
    def evaluate(self, state: Any, x: jnp.array) -> Tuple[jnp.array, Any]:
        return total_cost(x),state



In [15]:
import sys
sys.path.append('/home/cyh/evox/my_so_algorithm/')  # 添加包含my_pso.py文件的路径
from my_pso import my_PSO  # 导入my_PSO类

In [16]:
problem_params = {
    'Pd': 2350,
    'Hd': 1250,
    'chp_units': chp_units,
    'power_units': power_units,
    'heat_units': heat_units
}
# 初始化PSO优化器
algorithm = my_PSO(
    pop_size=1000,
    problem_type='chp_24_eld',
    problem_params=problem_params,
    init_method='uniform',
)


In [17]:
#创建一个problem实例
Problem = CHPELD_24()

monitor = monitors.EvalMonitor(full_fit_history=True)
# create a workflow
workflow = workflows.StdWorkflow(
    algorithm,
    Problem,
    monitors=[monitor],
)

In [18]:
length_HR = HRmax - HRmin
print(len(length_HR))
print(length_HR[3].dtype)



6
float32


In [19]:
length_PR = PRmax - PRmin
print(len(length_PR))
print(length_PR.shape)



6
(6,)


In [20]:
import time
generation_best_fitness = []  # 用于存储每代的最佳适应度
global_best_fitness = []      # 用于存储全局最佳适应度


# init the workflow
key = jax.random.PRNGKey(9)
state = workflow.init(key)

start_time = time.time()
# run the workflow 
for i in range(1000):
    state = workflow.step(state)
   

end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

Time taken: 13.554413795471191 seconds


In [21]:
monitor.get_best_fitness()

Array(57862.625, dtype=float32)

In [22]:
monitor.get_best_solution()

Array([628.31836, 224.39932, 299.19977, 159.73355,  60.     , 159.73314,
        60.     , 109.86776, 159.73288,  40.     ,  40.     ,  55.     ,
        55.     ,  81.01268,  40.05939,  92.93331,  40.     ,  10.     ,
        35.     , 104.80712,  75.05127, 111.4969 ,  75.     ,  40.     ,
        20.     , 463.63498,  60.     ,  60.     , 120.     , 120.     ],      dtype=float32)

In [23]:
p = monitor.get_best_solution()[:dim_P+dim_PH]
print(sum(p))





2349.9902


In [24]:
h = monitor.get_best_solution()[dim_P+dim_PH:]
print(sum(h))

1249.9902


In [25]:
pp = monitor.get_best_solution()[:13]
chp_p = monitor.get_best_solution()[13:19]
chp_h = monitor.get_best_solution()[19:25]
hh = monitor.get_best_solution()[25:30]

for i, value in enumerate(pp):
    if value < power_units['Pmin'][i] or value > power_units['Pmax'][i]:
        print(f"Value {value} at index {i} is out of bounds.")

for i, value in enumerate(chp_p):
    if value < chp_units['PRmin'][i] or value > chp_units['PRmax'][i]:
        print(f"Value {value} at index {i} is out of bounds.")

for i, value in enumerate(chp_h):
    if value < chp_units['HRmin'][i] or value > chp_units['HRmax'][i]:
        print(f"Value {value} at index {i} is out of bounds.")

for i, value in enumerate(hh):
    if value < heat_units['Hmin'][i] or value > heat_units['Hmax'][i]:
        print(f"Value {value} at index {i} is out of bounds.")



In [26]:
import jax
import jax.numpy as jnp

def check_constraints(array):
    """
    检查数组中的值是否满足特定的约束条件。
    参数：
        array: jax.numpy 数组，包含功率和热量的值。
    返回：
        constraints_satisfied: 一个布尔数组，表示每个 CHP 单元的约束是否被满足。
    """
    # 假设 array 的长度为 30，索引从 0 开始
    # 功率单元数量（Np） = 13
    # 联合热电机组数量（Nc） = 6
    Np = 13
    Nc = 6

    # 提取 CHP 单元的功率和热量输出
    p_array = array[Np:Np+Nc]          # 索引 13 到 18，共 6 个元素
    heat_array = array[Np+Nc:Np+2*Nc]  # 索引 19 到 24，共 6 个元素

    # 初始化约束满足情况的布尔数组
    constraints_satisfied = jnp.zeros(Nc, dtype=bool)

    # 定义每个 CHP 单元的约束检查函数
    def check_unit_constraints(idx, p, heat):
        """
        检查单个 CHP 单元的约束。
        参数：
            idx: 单元索引（0 到 5）
            p: 该单元的功率输出
            heat: 该单元的热量输出
        返回：
            布尔值，表示约束是否被满足
        """
        # 初始化热量的最小值和最大值
        heat_min = -jnp.inf
        heat_max = jnp.inf

        # CHP 单元 14 或 16（索引 0 或 2）
        if idx in [0, 2]:
            if (81 <= p) & (p <= 98.8):
                heat_min = 104.8 - (524/89)*(p - 81)
                heat_max = 104.8 + (188/335)*(p - 81)
            elif (98.8 < p) & (p <= 215):
                heat_min = 0.0
                heat_max = 104.8 + (188/335)*(p - 81)
            elif (215 < p) & (p <= 247):
                heat_min = 0.0
                heat_max = - (45/8)*(p - 247)
            else:
                return False  # 功率超出范围，约束不适用

        # CHP 单元 15 或 17（索引 1 或 3）
        elif idx in [1, 3]:
            if (40 <= p) & (p <= 44):
                heat_min = 75 - (591/40)*(p - 40)
                heat_max = 75 + (101/117)*(p - 40)
            elif (44 < p) & (p <= 110.2):
                heat_min = 0.0
                heat_max = 75 + (101/117)*(p - 40)
            elif (110.2 < p) & (p <= 125.8):
                heat_min = 0.0
                heat_max = 32.4 - (86/13)*(p - 125.8)
            else:
                return False

        # CHP 单元 18（索引 4）
        elif idx == 4:
            if (10 <= p) & (p <= 20):
                heat_min = 80 - 4*p
                heat_max = 40 + (3/7)*(p - 10)
            elif (20 < p) & (p <= 45):
                heat_min = 0.0
                heat_max = 40 + (3/7)*(p - 10)
            elif (45 < p) & (p <= 60):
                heat_min = 0.0
                heat_max = - (11/3)*(p - 60)
            else:
                return False

        # CHP 单元 19（索引 5）
        elif idx == 5:
            if (35 <= p) & (p <= 90):
                heat_min = 0.0
                heat_max = 20 + (5/11)*(p - 35)
            elif (90 < p) & (p <= 105):
                heat_min = 0.0
                heat_max = - (5/3)*(p - 105)
            else:
                return False

        else:
            return False  # 无效的单元索引

        # 检查热量是否在允许范围内
        if (heat >= heat_min) & (heat <= heat_max):
            return True
        else:
            return False

    # 对每个 CHP 单元进行约束检查
    for idx in range(Nc):
        p = p_array[idx]
        heat = heat_array[idx]
        is_satisfied = check_unit_constraints(idx, p, heat)
        constraints_satisfied = constraints_satisfied.at[idx].set(is_satisfied)

    return constraints_satisfied

# 示例数组
array = monitor.get_best_solution()

# 检查约束
constraints_satisfied = check_constraints(array)

print("每个 CHP 单元的约束满足情况：", constraints_satisfied)


每个 CHP 单元的约束满足情况： [ True  True  True  True  True  True]


In [27]:
power_error = sum(p) - Pd
heat_error = sum(h) - Hd
print(power_error)
print(heat_error)

power_diff = power_error / (dim_P + dim_PH)
heat_diff = heat_error / (dim_PH + dim_H)


x_final_power = monitor.get_best_solution()[:dim_P+dim_PH] - power_diff
x_final_heat = monitor.get_best_solution()[dim_P+dim_PH:] - heat_diff

print(f"调整后功率综合为{sum(x_final_power)}")
print(f"调整后热量综合为{sum(x_final_heat)}")

    

print(f"最终成本：{_total_cost(jnp.concatenate([x_final_power, x_final_heat]))}")


-0.009765625
-0.009765625
调整后功率综合为2349.999755859375
调整后热量综合为1250.0
最终成本：57862.875
